Technical Question Explainer

A tool that takes a technical question (or code snippet) and returns a structured explanation.  
Supports **OpenAI** and **Ollama** (local models) — switchable from the Gradio UI.

---
### Setup
```bash
uv add openai gradio
```
For **Ollama**, make sure it's running locally and you've pulled a model:
```bash
ollama pull llama3.2

In [2]:
import os
import gradio as gr
from openai import OpenAI

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL   = "gpt-4o-mini"


In [5]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
OLLAMA_MODEL    = "llama3.2" 

In [6]:
SYSTEM_PROMPT = """You are an expert technical educator.
When given a technical question or code snippet, provide:
1. A clear, concise explanation (2-3 sentences)
2. A simple analogy to make it relatable
3. A line-by-line or concept breakdown (especially for code)
4. Key takeaways as bullet points

Keep explanations accessible but accurate. Format your response in clean markdown."""

In [7]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [8]:
def get_client(provider: str) -> tuple[OpenAI, str]:
    """Return (OpenAI client, model name) for the chosen provider."""
    if provider == "Ollama (Local)":
        client = OpenAI(
            base_url=OLLAMA_BASE_URL,
            api_key="ollama",   
        )
        return client, OLLAMA_MODEL
    else:
        if not OPENAI_API_KEY :
            raise ValueError("Please set OPENAI_API_KEY in the Configuration cell.")
        return OpenAI(api_key=OPENAI_API_KEY), OPENAI_MODEL

In [10]:
def explain_streaming(question: str, provider: str):
    """Gradio streaming generator — yields progressively longer markdown strings."""
    if not question.strip():
        yield "Please enter a question or code snippet."
        return

    try:
        client, model = get_client(provider)
    except ValueError as e:
        yield f" **Error:** {e}"
        return

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": question},
    ]

    accumulated = f"*Using **{model}** via {provider}…*\n\n---\n\n"
    try:
        with client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True,
        ) as stream:
            for chunk in stream:
                delta = chunk.choices[0].delta.content or ""
                accumulated += delta
                yield accumulated
    except Exception as e:
        yield f"**API Error:** {e}"

In [11]:
EXAMPLE_QUESTION = question.strip()

with gr.Blocks(
    title="Tech Explainer",
    theme=gr.themes.Base(
        primary_hue="teal",
        neutral_hue="zinc",
        font=[gr.themes.GoogleFont("IBM Plex Mono"), "monospace"],
    ),
    css="""
        #header { text-align:center; padding:1rem 0 0.25rem; }
        #sub    { text-align:center; color:#71717a; margin-bottom:1.5rem; }
        footer  { display:none !important; }
    """,
) as demo:

    gr.Markdown("#Technical Question Explainer", elem_id="header")
    gr.Markdown(
        "Paste any code snippet or technical question — get a structured explanation instantly.",
        elem_id="sub",
    )

    with gr.Row(equal_height=True):
        with gr.Column(scale=3):
            question_box = gr.Textbox(
                label="Question or Code Snippet",
                placeholder="Paste code or ask a technical question…",
                lines=6,
                value=EXAMPLE_QUESTION,
            )
            with gr.Row():
                submit_btn = gr.Button("Explain", variant="primary", scale=4)
                clear_btn  = gr.Button("Clear",  variant="secondary", scale=1)

        with gr.Column(scale=1):
            provider_radio = gr.Radio(
                choices=["OpenAI (GPT-4o-mini)", "Ollama (Local)"],
                value="OpenAI (GPT-4o-mini)",
                label="Provider",
            )
            gr.Markdown(
                """**OpenAI** — set `OPENAI_API_KEY` in cell 3.\n\n"""
                """**Ollama** — needs [Ollama](https://ollama.com) running locally.  \n"""
                """`ollama pull llama3.2`"""
            )

    answer_box = gr.Markdown(
        label="Explanation",
        value="*Your explanation will appear here…*",
    )

    gr.Examples(
        examples=[
            [EXAMPLE_QUESTION],
            ["What is the difference between a process and a thread?"],
            ["Explain Python's GIL and when it actually matters."],
            ["What is gradient descent and how does backpropagation use it?"],
            ["What does async/await do under the hood in Python?"],
            ["Explain the CAP theorem in distributed systems."],
        ],
        inputs=question_box,
        label="💡 Example Questions — click to load",
    )

    submit_btn.click(explain_streaming, [question_box, provider_radio], answer_box)
    question_box.submit(explain_streaming, [question_box, provider_radio], answer_box)
    clear_btn.click(
        lambda: (EXAMPLE_QUESTION, "*Your explanation will appear here…*"),
        outputs=[question_box, answer_box],
    )



In [12]:
demo.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
